In [6]:
from google.colab import drive
drive.mount("/content/drive")
DATA_PATH = "/content/drive/MyDrive/NLP_ReAct_RAG_Project/pdfs"

PDF_FORM = f"{DATA_PATH}/on-bilgilendirme-formu.pdf"
PDF_ACAD = f"{DATA_PATH}/cayma_hakki_akademik.pdf"

print(PDF_FORM)
print(PDF_ACAD)

import os
os.makedirs(DATA_PATH, exist_ok=True)

print("DATA_PATH:", DATA_PATH)
print("Beklenen dosyalar:\n", PDF_FORM, "\n", PDF_ACAD)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP_ReAct_RAG_Project/pdfs/on-bilgilendirme-formu.pdf
/content/drive/MyDrive/NLP_ReAct_RAG_Project/pdfs/cayma_hakki_akademik.pdf
DATA_PATH: /content/drive/MyDrive/NLP_ReAct_RAG_Project/pdfs
Beklenen dosyalar:
 /content/drive/MyDrive/NLP_ReAct_RAG_Project/pdfs/on-bilgilendirme-formu.pdf 
 /content/drive/MyDrive/NLP_ReAct_RAG_Project/pdfs/cayma_hakki_akademik.pdf


2:

In [7]:
import shutil, os

SRC_FORM = "/mnt/data/on-bilgilendirme-formu.pdf"
SRC_ACAD = "/mnt/data/cayma_hakki_akademik.pdf"

if os.path.exists(SRC_FORM) and not os.path.exists(PDF_FORM):
    shutil.copy(SRC_FORM, PDF_FORM)
    print("✅ Kopyalandı:", PDF_FORM)
else:
    print("ℹ️ Zaten var ya da kaynak yok:", PDF_FORM)

if os.path.exists(SRC_ACAD) and not os.path.exists(PDF_ACAD):
    shutil.copy(SRC_ACAD, PDF_ACAD)
    print("✅ Kopyalandı:", PDF_ACAD)
else:
    print("ℹ️ Zaten var ya da kaynak yok:", PDF_ACAD)


ℹ️ Zaten var ya da kaynak yok: /content/drive/MyDrive/NLP_ReAct_RAG_Project/pdfs/on-bilgilendirme-formu.pdf
ℹ️ Zaten var ya da kaynak yok: /content/drive/MyDrive/NLP_ReAct_RAG_Project/pdfs/cayma_hakki_akademik.pdf


3.

In [8]:
!pip -q install groq

from google.colab import userdata
from groq import Groq
import groq

print("groq version:", groq.__version__)

groq_api_key = userdata.get("GROQ_API_KEY")
client = Groq(api_key=groq_api_key)

# model bağlantı testi
chat_completion = client.chat.completions.create(
    messages=[{"role": "user", "content": "Hello, how are you?"}],
    model="llama-3.3-70b-versatile",
    temperature=0
)
print(chat_completion.choices[0].message.content)


groq version: 1.0.0
Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have. How can I help you today?


4

In [9]:
!pip -q install pymupdf

import fitz  # PyMuPDF

def extract_pages_from_pdf(pdf_path: str):
    doc = fitz.open(pdf_path)
    pages = []
    for i in range(len(doc)):
        text = doc[i].get_text("text").strip()
        if text:
            pages.append({"page": i+1, "text": text})
    return pages

form_pages = extract_pages_from_pdf(PDF_FORM)
acad_pages = extract_pages_from_pdf(PDF_ACAD)

print("Ön bilgilendirme formu sayfa:", len(form_pages))
print("Akademik makale sayfa:", len(acad_pages))

print("\n--- FORM (ilk sayfa ilk 600 karakter) ---\n", form_pages[0]["text"][:600])
print("\n--- ACAD (ilk sayfa ilk 600 karakter) ---\n", acad_pages[0]["text"][:600])


Ön bilgilendirme formu sayfa: 6
Akademik makale sayfa: 24

--- FORM (ilk sayfa ilk 600 karakter) ---
 ÖN BİLGİLENDİRME FORMU 
1- TARAFLAR VE KONU 
İşbu Ön Bilgilendirme Formu’nun konusu, internet ürün satın alacak kişi ("ALICI") ile Büyükdere Cad. 
Ali Kaya Sok. No:5 Levent 34394 İstanbul adresinde bulunan eip Eczacıbaşı İlaç Pazarlama Anonim 
Şirketi ("SATICI") arasındaki mesafeli satış sözleşmesine ilişkin tüketicilerin Tüketicilerin Korunması 
Hakkındaki Kanun ve Mesafeli Sözleşmeler Yönetmeliği hükümleri uyarınca bilgilendirilmesidir.  
2- SATICI ve ALICI BİLGİLERİ 
SATICI: 
Ünvanı: EİP ECZACIBAŞI İLAÇ PAZARLAMA A.Ş. 
MERSİS No. 0326001967700012 
Adresi: Büyükdere Caddesi. Ali Kaya Sokak. 

--- ACAD (ilk sayfa ilk 600 karakter) ---
 667
TAAD, Yıl:7, Sayı:27 (Temmuz 2016)
MESAFELİ SÖZLEŞMELERDE TÜKETİCİNİN CAYMA HAKKI
Consumer’s Right to Withdrawal in Distance Selling Contracts
Arş. Gör. Dr. Nafiye YÜCEDAĞ GÖZTEPE*
ÖZET
Tarafların yüz yüze gelmeden kurduğu mesafeli sözleşmelerde, sö

5

In [10]:
def chunk_text(text: str, max_chars=1200, min_chars=200):
    # Paragraflara göre böl, sonra max_chars'ı aşınca yeni chunk aç
    paras = [p.strip() for p in text.split("\n") if p.strip()]
    chunks = []
    buf = ""
    for p in paras:
        if len(buf) + len(p) + 1 <= max_chars:
            buf = (buf + "\n" + p).strip()
        else:
            if len(buf) >= min_chars:
                chunks.append(buf)
            buf = p
    if len(buf) >= min_chars:
        chunks.append(buf)
    return chunks

def build_chunks_from_pages(pages, source_name: str):
    all_chunks = []
    for item in pages:
        page_no = item["page"]
        text = item["text"]
        # Sayfa çok uzunsa böl
        page_chunks = chunk_text(text, max_chars=1200, min_chars=200)
        for idx, ch in enumerate(page_chunks, start=1):
            all_chunks.append({
                "content": ch,
                "source": source_name,
                "page": page_no,
                "chunk_id": idx
            })
    return all_chunks

chunks_form = build_chunks_from_pages(form_pages, "on-bilgilendirme-formu.pdf")
chunks_acad = build_chunks_from_pages(acad_pages, "cayma_hakki_akademik.pdf")

chunks = chunks_form + chunks_acad

print("Chunk sayısı (form):", len(chunks_form))
print("Chunk sayısı (akad):", len(chunks_acad))
print("Toplam chunk:", len(chunks))

print("\nÖrnek chunk:\n", chunks[0]["source"], "Sayfa:", chunks[0]["page"])
print(chunks[0]["content"][:700])


Chunk sayısı (form): 17
Chunk sayısı (akad): 69
Toplam chunk: 86

Örnek chunk:
 on-bilgilendirme-formu.pdf Sayfa: 1
ÖN BİLGİLENDİRME FORMU
1- TARAFLAR VE KONU
İşbu Ön Bilgilendirme Formu’nun konusu, internet ürün satın alacak kişi ("ALICI") ile Büyükdere Cad.
Ali Kaya Sok. No:5 Levent 34394 İstanbul adresinde bulunan eip Eczacıbaşı İlaç Pazarlama Anonim
Şirketi ("SATICI") arasındaki mesafeli satış sözleşmesine ilişkin tüketicilerin Tüketicilerin Korunması
Hakkındaki Kanun ve Mesafeli Sözleşmeler Yönetmeliği hükümleri uyarınca bilgilendirilmesidir.
2- SATICI ve ALICI BİLGİLERİ
SATICI:
Ünvanı: EİP ECZACIBAŞI İLAÇ PAZARLAMA A.Ş.
MERSİS No. 0326001967700012
Adresi: Büyükdere Caddesi. Ali Kaya Sokak. No:5 Levent/İstanbul
Telefon: 0212 350 80 00
Fax:  0212 350 82 28
ALICI:
Adı- Soyadı:
E-mail:
Tarih:
3- ÜRÜN AÇ


6

In [11]:
import pickle

save_path = "/content/drive/MyDrive/ReAct/pdf_chunks.pkl"
with open(save_path, "wb") as f:
    pickle.dump(chunks, f)

print("✅ PDF chunk’ları Drive’a kaydedildi:", save_path)


✅ PDF chunk’ları Drive’a kaydedildi: /content/drive/MyDrive/ReAct/pdf_chunks.pkl


7

In [12]:
import re
import math
from collections import Counter

class SimpleRAG:
    def __init__(self):
        self.vector_store = []

    def get_embedding(self, text):
        words = re.findall(r'\w+', text.lower())
        return Counter(words)

    def _cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum(vec1[x] * vec2[x] for x in intersection)
        sum1 = sum(v*v for v in vec1.values())
        sum2 = sum(v*v for v in vec2.values())
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        return float(numerator) / denominator if denominator else 0.0

    def ingest(self, chunk_obj: dict):
        content = chunk_obj["content"]
        self.vector_store.append({
            "content": content,
            "vector": self.get_embedding(content),
            "source": chunk_obj["source"],
            "page": chunk_obj["page"],
            "chunk_id": chunk_obj["chunk_id"],
        })

    def retrieve(self, query, top_k=3, source_filter=None):
        query_vec = self.get_embedding(query)
        scores = []

        for item in self.vector_store:
            if source_filter and item["source"] not in source_filter:
                continue
            score = self._cosine_similarity(query_vec, item["vector"])
            scores.append((score, item))

        scores.sort(key=lambda x: x[0], reverse=True)
        top = [it for it in scores[:top_k] if it[0] > 0]

        # Ham context: her parça Kaynak+Sayfa ile gelsin
        out = []
        for score, item in top:
            header = f"[{item['source']} | Sayfa {item['page']}]"
            out.append(header + "\n" + item["content"])
        return "\n\n---\n\n".join(out)

rag_engine = SimpleRAG()
for ch in chunks:
    rag_engine.ingest(ch)

print("✅ RAG'e yüklenen toplam chunk:", len(rag_engine.vector_store))


✅ RAG'e yüklenen toplam chunk: 86


8

In [13]:
def calculator(expression: str) -> str:
    """
    Calculates the result of a mathematical expression.
    Used only when a numerical calculation is required.
    Example: calculator: 2 * 52
    """
    try:
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Hesaplama hatası: geçersiz karakter."
        return str(eval(expression))
    except Exception as e:
        return f"Hesaplama hatası: {e}"

def ecommerce_policy_search_tool(query: str) -> str:
    """
    Searches inside uploaded PDFs:
    - on-bilgilendirme-formu.pdf
    - cayma_hakki_akademik.pdf

    Returns RAW_CONTEXT with source+page.
    Do not use for casual chat.
    """
    print(f"\n🔍 [PDF ARAMASI] '{query}' için taranıyor...")

    # İstersen burada query'ye göre source_filter yapabilirsin:
    # Örn: sadece "akademik" yazarsa akademik pdf'e bak.
    source_filter = None
    if "akademik" in query.lower():
        source_filter = ["cayma_hakki_akademik.pdf"]
    if "form" in query.lower() or "ön bilgilendirme" in query.lower():
        source_filter = ["on-bilgilendirme-formu.pdf"]

    result = rag_engine.retrieve(query, top_k=3, source_filter=source_filter)

    if not result:
        return "PDF’lerde bu konuya ilişkin açık bir ifade bulunmamaktadır."
    return result

tools = [calculator, ecommerce_policy_search_tool]
known_actions = {tool.__name__: tool for tool in tools}

tools_with_desc = "\n\n".join(
    [tool.__name__ + (tool.__doc__ or "") for tool in tools]
)

print("✅ Tools hazır:", list(known_actions.keys()))


✅ Tools hazır: ['calculator', 'ecommerce_policy_search_tool']


9

In [14]:
system_prompt = f"""
You are a ReAct Legal AI Assistant specialized in Turkish distance sales and the right of withdrawal.

You must follow this strict loop:
Thought -> Action -> PAUSE -> Observation -> Answer

GUIDELINES:
1. Use 'ecommerce_policy_search_tool' to retrieve relevant RAW_CONTEXT from the PDFs.
2. The tool returns RAW legal context with [PDF | Page], not final answers.
3. Do NOT invent rules. If PDFs don't contain the answer, say so clearly.
4. Always answer in Turkish.
5. In the final Answer, ALWAYS cite the PDF name and page number(s) you used.
6. Use 'calculator' ONLY if a numerical calculation is required.

Your available actions are:

{tools_with_desc}

Example:

Question: Ön bilgilendirme formuna göre cayma hakkı süresi kaç gündür?
Thought: Bu bilgi form PDF'de geçer, tool ile ilgili kısmı bulmalıyım.
Action: ecommerce_policy_search_tool: cayma hakkı süresi
PAUSE
Observation: [on-bilgilendirme-formu.pdf | Sayfa 2] ... 14 gün ...
Answer: Cayma hakkı süresi 14 gündür. Kaynak: on-bilgilendirme-formu.pdf, Sayfa: 2
""".strip()

print(system_prompt[:900])


You are a ReAct Legal AI Assistant specialized in Turkish distance sales and the right of withdrawal.

You must follow this strict loop:
Thought -> Action -> PAUSE -> Observation -> Answer

GUIDELINES:
1. Use 'ecommerce_policy_search_tool' to retrieve relevant RAW_CONTEXT from the PDFs.
2. The tool returns RAW legal context with [PDF | Page], not final answers.
3. Do NOT invent rules. If PDFs don't contain the answer, say so clearly.
4. Always answer in Turkish.
5. In the final Answer, ALWAYS cite the PDF name and page number(s) you used.
6. Use 'calculator' ONLY if a numerical calculation is required.

Your available actions are:

calculator
    Calculates the result of a mathematical expression.
    Used only when a numerical calculation is required.
    Example: calculator: 2 * 52
    

ecommerce_policy_search_tool
    Searches inside uploaded PDFs:
    - on-bilgilendirme-formu.pdf
  


10

In [36]:

# class Agent:
#     def __init__(self, system="", model_name="llama-3.3-70b-versatile"):
#         self.system = system
#         self.model_name = model_name
#         self.messages = []
#         if self.system:
#             self.messages.append({"role": "system", "content": system})

#     def __call__(self, message):
#         self.messages.append({"role": "user", "content": message})
#         result = self.execute()
#         self.messages.append({"role": "assistant", "content": result})
#         return result

#     def execute(self):
#         result = client.chat.completions.create(
#             model=self.model_name,
#             messages=self.messages,
#             temperature=0,
#             stop=["PAUSE", "Observation:", "<|"]
#         )
#         return result.choices[0].message.content

class Agent:
    def __init__(self, system="", model_name="llama-3.1-8b-instant", provider="groq"):
        self.system = system
        self.model_name = model_name
        self.provider = provider
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        if self.provider == "groq":
            result = client.chat.completions.create(
                model=self.model_name,
                messages=self.messages,
                temperature=0,
                stop=["PAUSE", "Observation:", "<|"]
            )
            return result.choices[0].message.content

        elif self.provider == "gemini":
            # Gemini stop token desteği aynı değil; biz prompt ile disipline ediyoruz
            return call_gemini(self.messages, model=self.model_name)

        else:
            raise ValueError("Unknown provider: " + self.provider)


11

In [40]:

# def run_agent_once(question: str, model_name: str, max_turns=4):
#     agent = Agent(system_prompt, model_name=model_name)
#     next_prompt = question
#     full_trace = []

#     for turn in range(max_turns):
#         result = agent(next_prompt)
#         full_trace.append(result)

#         actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
#         if actions:
#             action, action_input = actions[0].groups()
#             observation = known_actions[action](action_input)
#             full_trace.append("Observation: " + observation)

#             next_prompt = (
#                 f"Observation: {observation}\n\n"
#                 "Şimdi sadece final cevap ver: 'Answer:' ile başla, "
#                 "ve mutlaka PDF adı + sayfa yaz."
#             )
#         else:
#             return "\n".join(full_trace)

#     full_trace.append("Answer: (Model final yanıt üretmedi)")
#     return "\n".join(full_trace)
def run_agent_once(question: str, model_name: str, provider: str, max_turns=4):
    agent = Agent(system_prompt, model_name=model_name, provider=provider)
    next_prompt = question
    full_trace = []

    for turn in range(max_turns):
        result = agent(next_prompt)
        full_trace.append(result)

        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            observation = known_actions[action](action_input)
            full_trace.append("Observation: " + observation)

            next_prompt = (
                f"Observation: {observation}\n\n"
                "Şimdi sadece final cevap ver: 'Answer:' ile başla, "
                "ve mutlaka PDF adı + sayfa yaz."
            )
        else:
            return "\n".join(full_trace)

    full_trace.append("Answer: (Model final yanıt üretmedi)")
    return "\n".join(full_trace)



12

In [26]:
query("Ön bilgilendirme formuna göre cayma hakkı süresi kaç gündür? Kaynak PDF ve sayfayı yaz.")



QUESTION: Ön bilgilendirme formuna göre cayma hakkı süresi kaç gündür? Kaynak PDF ve sayfayı yaz.
Thought: Bu bilgi form PDF'de geçer, tool ile ilgili kısmı bulmalıyım.
Action: ecommerce_policy_search_tool: cayma hakkı süresi

=> Running ecommerce_policy_search_tool(cayma hakkı süresi)

🔍 [PDF ARAMASI] 'cayma hakkı süresi' için taranıyor...
Observation: [cayma_hakki_akademik.pdf | Sayfa 7]
bakımından tüketicinin hakkını kötüye kullandığına ilişkin bir değerlendirme
yapılması ile menfaatler dengesine uygun bir sonuca ulaşılabileceği düşünülse
de, satıcının ön bilgilendirme yükümlülüğünü yerine getirerek cayma süresi-
nin uzamasını engellemesinin kolaylıkla mümkün olduğu göz önünde tutul-
malıdır18. O halde, satıcı cayma hakkına ilişkin usulüne uygun bilgilendirme
yapmamış olmasının sonuçlarına katlanmalı, tüketicinin cayma hakkının bu-
lunduğundan haberdar olması dikkate alınmamalıdır. Bu nedenle evine yeni
bir çamaşır makinesi almış olan tüketici, kendisine cayma hakkının bulundu-
ğu 

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01kdjw55xje95bz06v1akcf2pd` service tier `on_demand` on tokens per day (TPD): Limit 100000, Used 99286, Requested 1579. Please try again in 12m27.36s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

13

In [18]:
query("""Ön bilgilendirme formunda:
- Satıcı kaç gün içinde iade yapar?
- Tüketici kaç gün içinde ürünü geri gönderir?
Toplamı hesapla.
Cevapta Kaynak PDF ve sayfayı yaz.""")



QUESTION: Ön bilgilendirme formunda:
- Satıcı kaç gün içinde iade yapar?
- Tüketici kaç gün içinde ürünü geri gönderir?
Toplamı hesapla.
Cevapta Kaynak PDF ve sayfayı yaz.
Thought: Bu bilgiler ön bilgilendirme formu PDF'de geçer, tool ile ilgili kısmı bulmalıyım.
Action: ecommerce_policy_search_tool: satıcı iade süresi, tüketici ürün geri gönderme süresi

=> Running ecommerce_policy_search_tool(satıcı iade süresi, tüketici ürün geri gönderme süresi)

🔍 [PDF ARAMASI] 'satıcı iade süresi, tüketici ürün geri gönderme süresi' için taranıyor...
Observation: [cayma_hakki_akademik.pdf | Sayfa 10]
ile ilgili doğrudan masraflara tüketicinin katlanması gerektiği düzenlenmiş-
tir28. Satıcı ön bilgilendirmede geri gönderme masraflarına tüketicinin katla-
nacağını bildirdiyse, tüketici geri gönderme masraflarını ödeme borcu altında
olacaktır. Bunun tam aksine Yönetmelik m.12(3) hükmüne göre ise, satıcı ön
bilgilendirmede geri gönderme için taşıyıcı belirttiyse malın belirtilen taşıyıcı
aracılığıyl

14

In [19]:
import json, re
from datetime import datetime

def norm(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    s = s.replace("’","'").replace("“","").replace("”","")
    return s

# 40 dataset-içi + 10 reasoning
BENCHMARK = [
  # ---------- DATASET-İÇİ (A1-A40) ----------
  {"id":"A1","type":"dataset","question":"Ön bilgilendirme formuna göre cayma hakkı süresi kaç gündür? Kaynak PDF ve sayfayı yaz.","expected":["14"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A2","type":"dataset","question":"Ön bilgilendirme formuna göre mal satışında cayma süresi hangi tarihten itibaren başlar? Kaynak PDF ve sayfayı yaz.","expected":["teslim","teslim tarih"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A3","type":"dataset","question":"Ön bilgilendirme formuna göre hizmet sözleşmelerinde cayma süresi ne zaman başlar? Kaynak PDF ve sayfayı yaz.","expected":["imzaland","sözleşmenin imzaland"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A4","type":"dataset","question":"Ön bilgilendirme formuna göre cayma bildirimi nereye yöneltilmelidir? Kaynak PDF ve sayfayı yaz.","expected":["satıcı","adres"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A5","type":"dataset","question":"Ön bilgilendirme formuna göre tüketici cayma hakkını kullanırken gerekçe göstermek zorunda mı? Kaynak PDF ve sayfayı yaz.","expected":["gerekçe göstermeksizin","hiçbir gerekçe"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A6","type":"dataset","question":"Ön bilgilendirme formuna göre cayma hakkı kullanıldığında satıcı bedeli en geç kaç gün içinde iade eder? Kaynak PDF ve sayfayı yaz.","expected":["14"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A7","type":"dataset","question":"Ön bilgilendirme formuna göre tüketici cayma hakkını kullandıktan sonra ürünü kaç gün içinde iade eder? Kaynak PDF ve sayfayı yaz.","expected":["10"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A8","type":"dataset","question":"Ön bilgilendirme formuna göre cayma hakkının kullanımından kaynaklanan masraflar kime aittir? Kaynak PDF ve sayfayı yaz.","expected":["satıcı"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A9","type":"dataset","question":"Ön bilgilendirme formuna göre cayma hakkı kullanılırken hukuki ve cezai sorumluluk üstlenilir mi? Kaynak PDF ve sayfayı yaz.","expected":["üstlenmeksizin","sorumluluk üstlenmeksizin"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A10","type":"dataset","question":"Ön bilgilendirme formuna göre mal teslim edilmeden önce cayma hakkı kullanılabilir mi? Kaynak PDF ve sayfayı yaz.","expected":["teslim"],"must_cite":["on-bilgilendirme-formu.pdf"]},

  # Cayma hakkı olmayan haller (form sayfa 2-3)
  {"id":"A11","type":"dataset","question":"Ön bilgilendirme formuna göre finansal piyasa dalgalanmasına bağlı ürün/hizmetlerde cayma hakkı var mı? Kaynak PDF ve sayfayı yaz.","expected":["cayma hakkı bulunmamaktadır","bulunmamaktadır"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A12","type":"dataset","question":"Ön bilgilendirme formuna göre tüketicinin istekleri doğrultusunda hazırlanan (kişiye özel) mallarda cayma hakkı var mı? Kaynak PDF ve sayfayı yaz.","expected":["cayma hakkı bulunmamaktadır","bulunmamaktadır"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A13","type":"dataset","question":"Ön bilgilendirme formuna göre çabuk bozulabilen veya SKT geçebilecek mallarda cayma hakkı var mı? Kaynak PDF ve sayfayı yaz.","expected":["cayma hakkı bulunmamaktadır","bulunmamaktadır"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A14","type":"dataset","question":"Ön bilgilendirme formuna göre ambalaj/bant/mühür açılmış ve hijyen açısından iadesi uygun olmayan mallarda cayma hakkı var mı? Kaynak PDF ve sayfayı yaz.","expected":["cayma hakkı bulunmamaktadır","bulunmamaktadır"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A15","type":"dataset","question":"Ön bilgilendirme formuna göre elektronik ortamda anında ifa edilen hizmetlerde cayma hakkı var mı? Kaynak PDF ve sayfayı yaz.","expected":["cayma hakkı bulunmamaktadır","bulunmamaktadır"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A16","type":"dataset","question":"Ön bilgilendirme formuna göre cayma süresi bitmeden tüketicinin onayı ile ifasına başlanan hizmetlerde cayma hakkı var mı? Kaynak PDF ve sayfayı yaz.","expected":["cayma hakkı bulunmamaktadır","bulunmamaktadır"],"must_cite":["on-bilgilendirme-formu.pdf"]},

  # iade için gerekli belgeler (form 6.4)
  {"id":"A17","type":"dataset","question":"Ön bilgilendirme formuna göre iade için hangi belgeler/unsurlar istenir? (en az 2 madde yaz) Kaynak PDF ve sayfayı yaz.","expected":["fatura","iade formu","kutusu","ambalaj"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A18","type":"dataset","question":"Ön bilgilendirme formuna göre kurumsal faturalı iadelerde ayrıca hangi belge gerekir? Kaynak PDF ve sayfayı yaz.","expected":["iade faturası"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A19","type":"dataset","question":"Ön bilgilendirme formuna göre ürünün değerinde azalma olursa tüketici ne ile yükümlü olur? Kaynak PDF ve sayfayı yaz.","expected":["kusuru oranında","tazmin"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A20","type":"dataset","question":"Ön bilgilendirme formuna göre kampanya limiti altına düşülürse ne olur? Kaynak PDF ve sayfayı yaz.","expected":["indirim","iptal"],"must_cite":["on-bilgilendirme-formu.pdf"]},

  # Teslimat (form sayfa 2)
  {"id":"A21","type":"dataset","question":"Ön bilgilendirme formuna göre ürün en geç kaç gün içinde teslim edilir? Kaynak PDF ve sayfayı yaz.","expected":["30"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A22","type":"dataset","question":"Ön bilgilendirme formuna göre olağanüstü durumlarda satıcı ne yapar? (kısaca) Kaynak PDF ve sayfayı yaz.","expected":["bilgilendirir"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A23","type":"dataset","question":"Ön bilgilendirme formuna göre alıcı teslim anında ürünü kontrol etmezse satıcı ne der? Kaynak PDF ve sayfayı yaz.","expected":["sorumluluk kabul etmeyecektir","sorumluluk kabul etmez"],"must_cite":["on-bilgilendirme-formu.pdf"]},

  # Akademik PDF’den doğrudan (tanım/süre/bilgilendirme)
  {"id":"A24","type":"dataset","question":"Akademik metne göre mesafeli sözleşme tanımı hangi unsurları içerir? (en az 2 unsur) Kaynak PDF ve sayfayı yaz.","expected":["eş zamanlı fiziksel varlığı olmaksızın","uzaktan iletişim araçları"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"A25","type":"dataset","question":"Akademik metne göre tüketici kaç gün içinde cayma hakkına sahiptir? Kaynak PDF ve sayfayı yaz.","expected":["14"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"A26","type":"dataset","question":"Akademik metne göre bilgilendirme yapılmazsa cayma hakkı süresi nasıl uzar? Kaynak PDF ve sayfayı yaz.","expected":["bir yıl","1 yıl"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"A27","type":"dataset","question":"Akademik metne göre cayma beyanı nasıl yöneltilmelidir? (en az 1 yöntem) Kaynak PDF ve sayfayı yaz.","expected":["yazılı","kalıcı veri saklayıcısı"],"must_cite":["cayma_hakki_akademik.pdf"]},

  # Kalan dataset-içi sorular (kısa/ölçülebilir)
  {"id":"A28","type":"dataset","question":"Ön bilgilendirme formuna göre kargo ücreti kaç TL olarak belirtilmiştir? Kaynak PDF ve sayfayı yaz.","expected":["6,99","6.99"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A29","type":"dataset","question":"Ön bilgilendirme formuna göre paketleme/kargo/teslim masrafları kimin tarafından karşılanır? Kaynak PDF ve sayfayı yaz.","expected":["alıcı"],"must_cite":["on-bilgilendirme-formu.pdf"]},

  # 11 tane daha kısa dataset-içi (A30-A40) -> çoğu form sayfa 2-3-5’ten
  {"id":"A30","type":"dataset","question":"Ön bilgilendirme formuna göre kredi kartı taksit detay planı kimin inisiyatifindedir? Kaynak PDF ve sayfayı yaz.","expected":["banka"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A31","type":"dataset","question":"Ön bilgilendirme formuna göre ürün teslim edilene kadar sorumluluk kime aittir? Kaynak PDF ve sayfayı yaz.","expected":["firmamıza","satıcı"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A32","type":"dataset","question":"Ön bilgilendirme formuna göre alıcı teslimatta adresinde yoksa firma edimini yerine getirmiş sayılır mı? Kaynak PDF ve sayfayı yaz.","expected":["yerine getirmiş","kabul edilecektir"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A33","type":"dataset","question":"Ön bilgilendirme formuna göre ürün kargoda bekleyip geri dönerse oluşan giderler kime aittir? Kaynak PDF ve sayfayı yaz.","expected":["alıcı"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A34","type":"dataset","question":"Ön bilgilendirme formuna göre tüketici hangi bilgileri okuyup onayladığını kabul eder? (en az 2 bilgi) Kaynak PDF ve sayfayı yaz.","expected":["teslimat","cayma hakkı","ödeme şekli","vergiler dahil"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A35","type":"dataset","question":"Ön bilgilendirme formuna göre banka/finans kuruluşu satıcıya ödeme yapmazsa ürün kaç gün içinde iade edilmelidir? Kaynak PDF ve sayfayı yaz.","expected":["3"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A36","type":"dataset","question":"Ön bilgilendirme formuna göre kredi kartlı ödemelerde iade nasıl yapılır? Kaynak PDF ve sayfayı yaz.","expected":["kredi kartına","iade sureti ile"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"A37","type":"dataset","question":"Akademik metne göre cayma beyanının satıcıya ulaşması şart mı, yoksa gönderilmesi yeterli mi? Kaynak PDF ve sayfayı yaz.","expected":["gönderilmesi yeterlidir","ulaşması aranmayacaktır"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"A38","type":"dataset","question":"Akademik metne göre sadece ürünü geri göndermek cayma hakkını kullanmak için yeterli midir? Kaynak PDF ve sayfayı yaz.","expected":["yeterli olmaz","açık bildirimi"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"A39","type":"dataset","question":"Akademik metne göre malın satıcı tarafından taşıyıcıya teslimi tüketiciye teslim sayılır mı? Kaynak PDF ve sayfayı yaz.","expected":["kabul edilmez","teslim olarak"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"A40","type":"dataset","question":"Akademik metne göre tüketici ayıplı malda cayma hakkı ile hangi hak arasında seçim yapabilir? Kaynak PDF ve sayfayı yaz.","expected":["sözleşmeden dönme","cayma hakkı"],"must_cite":["cayma_hakki_akademik.pdf"]},

  # ---------- REASONING (B41-B50) ----------
  {"id":"B41","type":"reasoning","question":"Cayma hakkı olmayan bir üründe satıcı iade kabul ederse bu tüketici lehine bağlayıcı olur mu? Gerekçeli cevap ver ve Kaynak PDF/sayfa yaz.","expected":["evet","tüketici lehine"],"must_cite":["on-bilgilendirme-formu.pdf","cayma_hakki_akademik.pdf"]},
  {"id":"B42","type":"reasoning","question":"Cayma süresi geçmişse tüketici yine de iade talep edebilir mi? Hangi istisnalar olabilir? Kaynak PDF/sayfa yaz.","expected":["ayıplı","bilgilendirme"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"B43","type":"reasoning","question":"Ön bilgilendirme yapılmazsa cayma hakkı süresi nasıl etkilenir? Kaynak PDF/sayfa yaz.","expected":["bir yıl","1 yıl"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"B44","type":"reasoning","question":"Tüketici ürünü kullandıysa cayma hakkını tamamen kaybeder mi? Ne olur? Kaynak PDF/sayfa yaz.","expected":["değer azalması","sorumlu"],"must_cite":["on-bilgilendirme-formu.pdf","cayma_hakki_akademik.pdf"]},
  {"id":"B45","type":"reasoning","question":"Satıcı iade süresini tek taraflı uzatabilir mi? Hangi yönde uzatabilir? Kaynak PDF/sayfa yaz.","expected":["tüketici lehine","uzatabilir"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"B46","type":"reasoning","question":"Formda 14 gün satıcı iade + 10 gün tüketici iade süresi varsa toplam süreç kaç gündür? Hesapla ve Kaynak PDF/sayfa yaz.","expected":["24"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"B47","type":"reasoning","question":"Cayma hakkı olmayan bir ürün ayıplı çıkarsa tüketici hangi haklara dayanabilir? Kaynak PDF/sayfa yaz.","expected":["ayıplı","sözleşmeden dönme"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"B48","type":"reasoning","question":"Dijital içerikte (anında ifa/teslim) cayma hakkı neden sınırlıdır? Kaynak PDF/sayfa yaz.","expected":["anında ifa","gayri maddi"],"must_cite":["on-bilgilendirme-formu.pdf"]},
  {"id":"B49","type":"reasoning","question":"Ön bilgilendirme hiç yoksa sözleşme geçersiz mi olur, yoksa tüketici lehine hangi sonuç doğar? Kaynak PDF/sayfa yaz.","expected":["cayma süresi uzar","bir yıl"],"must_cite":["cayma_hakki_akademik.pdf"]},
  {"id":"B50","type":"reasoning","question":"Bu düzenlemeler AB tüketici yaklaşımıyla kıyaslanınca iade masrafı bakımından temel fark nedir? Kaynak PDF/sayfa yaz.","expected":["yönerge","masraf","tüketici","satıcı"],"must_cite":["cayma_hakki_akademik.pdf"]},
]

print("✅ Benchmark hazır:", len(BENCHMARK), "soru")


✅ Benchmark hazır: 50 soru


15

In [27]:
# class Agent:
#     def __init__(self, system="", model_name="llama-3.3-70b-versatile"):
#         self.system = system
#         self.model_name = model_name
#         self.messages = []
#         if self.system:
#             self.messages.append({"role": "system", "content": system})

#     def __call__(self, message):
#         self.messages.append({"role": "user", "content": message})
#         result = self.execute()
#         self.messages.append({"role": "assistant", "content": result})
#         return result

#     def execute(self):
#         result = client.chat.completions.create(
#             model=self.model_name,
#             messages=self.messages,
#             temperature=0,
#             stop=["PAUSE", "Observation:", "<|"]
#         )
#         return result.choices[0].message.content

# def run_agent_once(question: str, model_name: str, max_turns=6):
#     agent = Agent(system_prompt, model_name=model_name)
#     next_prompt = question
#     full_trace = []

#     for _ in range(max_turns):
#         result = agent(next_prompt)
#         full_trace.append(result)

#         actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]
#         if actions:
#             action, action_input = actions[0].groups()
#             observation = known_actions[action](action_input)
#             next_prompt = f"Observation: {observation}"
#             full_trace.append("Observation: " + observation)
#         else:
#             # final
#             return "\n".join(full_trace)
#     return "\n".join(full_trace)
import re

NUM_WORDS = {
  "on dört":"14", "ondört":"14",
  "on":"10", "on gün":"10",
  "otuz":"30", "otuz gün":"30",
  "üç":"3", "üç gün":"3",
  "altı virgül doksan dokuz":"6,99"
}

def normalize_numbers_tr(text: str) -> str:
    t = text.lower().strip()
    for w, d in NUM_WORDS.items():
        t = t.replace(w, d)
    return t

def norm(s: str) -> str:
    s = s.lower().strip()
    s = re.sub(r"\s+", " ", s)
    return s

def contains_any(text: str, keywords: list[str]) -> bool:
    t = normalize_numbers_tr(text)
    return any(norm(k) in t for k in keywords)

def has_required_citation_flexible(text: str, must_cite: list[str]) -> bool:
    t = text.lower()
    aliases = {
      "on-bilgilendirme-formu.pdf": [
          "on-bilgilendirme-formu.pdf",
          "ön bilgilendirme formu",
          "on bilgilendirme formu"
      ],
      "cayma_hakki_akademik.pdf": [
          "cayma_hakki_akademik.pdf",
          "akademik",
          "cayma hakkı akademik"
      ]
    }
    for src in must_cite:
        if not any(a in t for a in aliases.get(src.lower(), [src.lower()])):
            return False
    return True

def score_item(item, model_output: str):
    ok_content = contains_any(model_output, item["expected"])
    ok_cite = has_required_citation_flexible(model_output, item["must_cite"])

    if item["type"] == "dataset":
        return 1.0 if (ok_content and ok_cite) else 0.0

    hit = sum(1 for k in item["expected"] if norm(k) in normalize_numbers_tr(model_output))
    if ok_content and ok_cite:
        return 1.0
    if ok_cite and hit >= max(1, len(item["expected"])//2):
        return 0.5
    return 0.0


16

In [37]:

# from groq import RateLimitError

# def safe_run(item, model_name):
#     try:
#         return run_agent_once(item["question"], model_name=model_name, max_turns=4)
#     except RateLimitError:
#         print("⚠️ Rate limit → küçük modele düşüyorum (llama-3.1-8b-instant)")
#         return run_agent_once(item["question"], model_name="llama-3.1-8b-instant", max_turns=3)
from groq import RateLimitError

def safe_run(item, model_name, provider):
    try:
        return run_agent_once(item["question"], model_name=model_name, provider=provider, max_turns=4)
    except RateLimitError:
        print("⚠️ Groq rate limit → küçük modele düşüyorum")
        return run_agent_once(item["question"], model_name="llama-3.1-8b-instant", provider="groq", max_turns=3)



17

In [38]:

import os
import pandas as pd
from datetime import datetime

# def run_benchmark(models: dict, out_dir="/content/drive/MyDrive/NLP_ReAct_RAG_Project/results"):
#     os.makedirs(out_dir, exist_ok=True)
#     timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

#     all_rows = []

#     for model_label, model_name in models.items():
#         print("\n" + "="*70)
#         print(f"▶ Model: {model_label} ({model_name})")
#         print("="*70)

#         for item in BENCHMARK:
#             trace = safe_run(item, model_name)  # ✅ burada safe_run
#             sc = score_item(item, trace)        # ✅ score_item h15’ten geliyor

#             all_rows.append({
#                 "model_label": model_label,
#                 "model_name": model_name,
#                 "id": item["id"],
#                 "type": item["type"],
#                 "question": item["question"],
#                 "score": sc,
#                 "trace": trace[:8000]
#             })
#             print(f"{item['id']} | {item['type']} | score={sc}")

#     df = pd.DataFrame(all_rows)

#     summary = df.groupby(["model_label","type"])["score"].agg(["count","mean","sum"]).reset_index()
#     overall = df.groupby(["model_label"])["score"].agg(["count","mean","sum"]).reset_index()

#     csv_path = f"{out_dir}/benchmark_results_{timestamp}.csv"
#     sum_path = f"{out_dir}/benchmark_summary_{timestamp}.csv"

#     df.to_csv(csv_path, index=False)
#     summary.to_csv(sum_path, index=False)

#     print("\n✅ Detay CSV:", csv_path)
#     print("✅ Özet CSV:", sum_path)

#     display(summary)
#     display(overall)
#     return df, summary, overall
def run_benchmark(models: dict, out_dir="/content/drive/MyDrive/NLP_ReAct_RAG_Project/results"):
    os.makedirs(out_dir, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    all_rows = []

    for model_label, cfg in models.items():
        provider = cfg["provider"]
        model_name = cfg["model"]
        print("\n" + "="*70)
        print(f"▶ Model: {model_label} ({provider} | {model_name})")
        print("="*70)

        for item in BENCHMARK:
            trace = safe_run(item, model_name, provider)
            sc = score_item(item, trace)
            all_rows.append({
                "model_label": model_label,
                "provider": provider,
                "model_name": model_name,
                "id": item["id"],
                "type": item["type"],
                "score": sc,
                "question": item["question"],
                "trace": trace[:8000]
            })
            print(f"{item['id']} | {item['type']} | score={sc}")

    df = pd.DataFrame(all_rows)
    summary = df.groupby(["model_label","type"])["score"].agg(["count","mean","sum"]).reset_index()
    overall = df.groupby(["model_label"])["score"].agg(["count","mean","sum"]).reset_index()

    csv_path = f"{out_dir}/benchmark_results_{timestamp}.csv"
    sum_path = f"{out_dir}/benchmark_summary_{timestamp}.csv"
    df.to_csv(csv_path, index=False)
    summary.to_csv(sum_path, index=False)

    print("\n✅ Detay CSV:", csv_path)
    print("✅ Özet CSV:", sum_path)
    display(summary); display(overall)
    return df, summary, overall



18

In [30]:
# models_to_test = {
#     "FREE_LLAMA_70B": "llama-3.3-70b-versatile",
#     # Groq hesabında açıksa alternatif ekle:
#     # "FREE_MIXTRAL": "mixtral-8x7b-32768",
# }

# df, summary, overall = run_benchmark(models_to_test)
models_to_test = {
    "FREE_SMALL": "llama-3.1-8b-instant",
    # hesabında açıksa alternatif:
    # "FREE_MIXTRAL": "mixtral-8x7b-32768",
}

df, summary, overall = run_benchmark(models_to_test)



▶ Model: FREE_SMALL (llama-3.1-8b-instant)

🔍 [PDF ARAMASI] 'cayma hakkı süresi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma hakkı süresi' için taranıyor...
A1 | dataset | score=0.0

🔍 [PDF ARAMASI] 'mal satışında cayma süresi başlama tarihi' için taranıyor...

🔍 [PDF ARAMASI] 'mal satışında cayma süresi başlama tarihi' için taranıyor...
A2 | dataset | score=1.0

🔍 [PDF ARAMASI] 'cayma hakkı başlama süresi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma hakkı başlama süresi' için taranıyor...
A3 | dataset | score=0.0

🔍 [PDF ARAMASI] 'cayma bildirimi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma bildirimi' için taranıyor...
A4 | dataset | score=1.0

🔍 [PDF ARAMASI] 'cayma hakkı gerekçesi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma hakkı gerekçesi' için taranıyor...
A5 | dataset | score=0.0

🔍 [PDF ARAMASI] 'cayma hakkı kullanıldığında satıcı bedeli iade süresi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma hakkı kullanıldığında satıcı bedeli iade süresi' için taranıyor...
A6 | dataset | score=1.0

🔍

,model_label,type,count,mean,sum
0,FREE_SMALL,dataset,40,0.725,29.0
1,FREE_SMALL,reasoning,10,0.400,4.0


,model_label,count,mean,sum
0,FREE_SMALL,50,0.66,33.0


In [34]:
!pip -q install -U google-genai


In [35]:
import os
from google import genai

# Colab > Secrets (🔑) içine GEMINI_API_KEY olarak ekle
gemini_key = userdata.get("GEMINI_API_KEY")
assert gemini_key, "GEMINI_API_KEY bulunamadı. Colab Secrets'e ekle."

gem_client = genai.Client(api_key=gemini_key)

def call_gemini(messages, model="gemini-1.5-pro"):
    # messages: [{"role":"system"|"user"|"assistant","content":"..."}]
    # Gemini tarafında content string list istenebiliyor; en güvenlisi tek string birleştirmek:
    prompt = ""
    for m in messages:
        role = m["role"]
        prompt += f"{role.upper()}:\n{m['content']}\n\n"

    resp = gem_client.models.generate_content(
        model=model,
        contents=prompt
    )
    return resp.text


In [41]:
models_to_test = {
    "FREE_LLAMA_8B": {"provider":"groq", "model":"llama-3.1-8b-instant"},
    "PAID_GEMINI_PRO": {"provider":"gemini", "model":"gemini-1.5-pro"},
}
df, summary, overall = run_benchmark(models_to_test)



▶ Model: FREE_LLAMA_8B (groq | llama-3.1-8b-instant)

🔍 [PDF ARAMASI] 'cayma hakkı süresi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma hakkı süresi' için taranıyor...
A1 | dataset | score=0.0

🔍 [PDF ARAMASI] 'mal satışında cayma süresi başlama tarihi' için taranıyor...

🔍 [PDF ARAMASI] 'mal satışında cayma süresi başlama tarihi' için taranıyor...
A2 | dataset | score=1.0

🔍 [PDF ARAMASI] 'cayma hakkı başlama süresi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma hakkı başlama süresi' için taranıyor...
A3 | dataset | score=0.0

🔍 [PDF ARAMASI] 'cayma bildirimi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma bildirimi' için taranıyor...
A4 | dataset | score=1.0

🔍 [PDF ARAMASI] 'cayma hakkı gerekçesi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma hakkı gerekçesi' için taranıyor...
A5 | dataset | score=0.0

🔍 [PDF ARAMASI] 'cayma hakkı kullanıldığında satıcı bedeli iade süresi' için taranıyor...

🔍 [PDF ARAMASI] 'cayma hakkı kullanıldığında satıcı bedeli iade süresi' için taranıyor...
A6 | dataset | sc

ClientError: 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemini-1.5-pro is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.', 'status': 'NOT_FOUND'}}

In [42]:
candidate_models = [
    "models/gemini-1.5-pro-latest",
    "models/gemini-1.5-flash-latest",
    "models/gemini-1.5-flash",
    "models/gemini-2.0-flash",
]

def test_gemini_model_name(m):
    try:
        r = gem_client.models.generate_content(
            model=m,
            contents="Sadece OK yaz."
        )
        print("✅ ÇALIŞTI:", m)
        return True
    except Exception as e:
        print("❌ olmadı:", m, "|", str(e)[:100])
        return False

working = None
for m in candidate_models:
    if test_gemini_model_name(m):
        working = m
        break

print("\nSEÇİLEN MODEL:", working)


❌ olmadı: models/gemini-1.5-pro-latest | 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemini-1.5-pro-latest is not found for API
❌ olmadı: models/gemini-1.5-flash-latest | 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemini-1.5-flash-latest is not found for A
❌ olmadı: models/gemini-1.5-flash | 404 NOT_FOUND. {'error': {'code': 404, 'message': 'models/gemini-1.5-flash is not found for API vers
❌ olmadı: models/gemini-2.0-flash | 429 RESOURCE_EXHAUSTED. {'error': {'code': 429, 'message': 'You exceeded your current quota, please 

SEÇİLEN MODEL: None
